# Fit $k_{ij}$  and $l_{ij}$ interactions parameters of Ethanol and Water

---
Let's call $\underline{\xi}$ the optimization parameters of a mixture. In order to optimize them, you need to provide experimental phase equilibria data. This can include VLE, LLE and VLLE data. The objective function used for each equilibria type are shown below:


### Vapor-Liquid Equilibria Data


$$ OF_{VLE}(\underline{\xi}) = w_y \sum_{j=1}^{Np} \left[ \sum_{i=1}^c (y_{i,j}^{cal} - y_{i,j}^{exp})^2 \right] +  w_P \sum_{j=1}^{Np} \left[ \frac{P_{j}^{cal} - P_{j}^{exp}}{P_{j}^{exp}} \right]^2$$

Where, $Np$ is the number of experimental data points, $y_i$ is the vapor molar fraction of the component $i$ and $P$ is the bubble pressure. The superscripts $cal$ and $exp$ refers to the computed and experimental values, respectively. Finally, $w_y$ is the weight for the vapor composition error and $w_P$ is the weight for the bubble pressure error.


### Liquid-Liquid Equilibria Data
$$ OF_{LLE}(\underline{\xi}) = w_x \sum_{j=1}^{Np} \sum_{i=1}^c  \left[x_{i,j} - x_{i,j}^{exp}\right]^2 +  w_w \sum_{j=1}^{Np} \sum_{i=1}^c \left[ w_{i,j} - w_{i,j}^{exp} \right]^2 $$

Where, $Np$ is the number of experimental data points, $x_i$ and $w_i$ are the molar fraction of the component $i$ on the liquids phases. The superscripts $cal$ and $exp$ refers to the computed and experimental values, respectively. Finally, $w_x$ and $w_w$ are the weights for the liquid 1 ($x$) and liquid 2 ($w$) composition error.


### Vapor-Liquid-Liquid Equilibria Data
$$ OF_{VLLE}(\underline{\xi}) = w_x \sum_{j=1}^{Np}  \sum_{i=1}^c  \left[x_{i,j}^{cal} - x_{i,j}^{exp}\right]^2 + w_w \sum_{j=1}^{Np}  \sum_{i=1}^c  \left[w_{i,j}^{cal} - w_{i,j}^{exp}\right]^2 +  w_y \sum_{j=1}^{Np}  \sum_{i=1}^c  \left[y_{i,j}^{cal} - y_{i,j}^{exp}\right]^2 + w_P \sum_{j=1}^{Np}  \left[ \frac{P_{j}^{cal}}{P_{j}^{exp}} - 1\right]^2 $$

Where, $Np$ is the number of experimental data points, $y_i$, $x_i$ and $w_i$ are the molar fraction of the component $i$ on the vapor and liquids phases, respectively. The superscripts $cal$ and $exp$ refers to the computed and experimental values, respectively. Finally, $w_x$ and $w_w$ are the weights for the liquid 1 ($x$) and liquid 2 ($w$) composition error, $w_y$ is the weight for vapor composition error and $w_P$ is weight for three phase equilibria pressure error.

If there is data for more than one equilibria type, the errors can be added accordinly. So the objective funcion becomes:

$$ OF(\underline{\xi}) =OF_{ELV}(\underline{\xi}) + OF_{ELL}(\underline{\xi}) + OF_{ELLV}(\underline{\xi})$$

---
This notebook has the purpose of showing how to optimize the $k_{ij}$ and $l_{ij}$ for a mixture in SGTPy. These parameters corrects the cross-interaction energy used for Mie potential interactions and the cross-interaction association energy:

$$\epsilon_{ij} = (1-k_{ij}) \frac{\sqrt{\sigma_i^3 \sigma_j^3}}{\sigma_{ij}^3} \sqrt{\epsilon_i \epsilon_j} ;\qquad \epsilon_{ij}^{AB} = (1 - l_{ij})\sqrt{\epsilon_{ii}^{AB} \epsilon_{jj}^{AB}}$$ 




First, it's needed to import the necessary modules

In [1]:
import numpy as np

from SGTPy import component, mixture, saftvrmie
from SGTPy.equilibrium import bubblePy
from SGTPy.fit import fit_asso

Now that the functions are available it is necessary to create the mixture.

In [2]:
water = component('water', ms = 1.7311, sigma = 2.4539 , eps = 110.85,
                    lambda_r = 8.308, lambda_a = 6.,  eAB = 1991.07, rcAB = 0.5624,
                    rdAB = 0.4, sites = [0,2,2], cii = 1.5371939421515455e-20)

ethanol = component('ethanol2C', ms = 1.7728, sigma = 3.5592 , eps = 224.50,
                    lambda_r = 11.319, lambda_a = 6., eAB = 3018.05, rcAB = 0.3547,
                    rdAB = 0.4, sites = [1,0,1], cii= 5.3141080872882285e-20)

mix = mixture(ethanol, water)

Now the experimental equilibria data is read and a tuple is created. It includes the experimental liquid composition, vapor composition, equilibrium temperature and pressure. This is done with ```datavle = (Xexp, Yexp, Texp, Pexp)```

In [3]:
# Experimental data obtained from So-Jin Park a,∗, Kyu-Jin Han a, J. Gmehling b

# Experimental temperature saturation in K
Texp = np.array([372.45, 370.05, 369.15, 369.15, 368.75, 367.95, 366.95, 366.65,
       366.05, 363.65, 363.65, 362.55, 361.55, 361.75, 360.35, 358.55,
       357.65, 357.15, 356.55, 356.15, 355.45, 355.15, 354.55, 354.65,
       354.35, 354.05, 353.65, 353.35, 353.15, 352.65, 351.95, 351.65,
       351.55, 351.45])

# Experimental pressure in Pa
Pexp = np.array([101300., 101300., 101300., 101300., 101300., 101300., 101300.,
       101300., 101300., 101300., 101300., 101300., 101300., 101300.,
       101300., 101300., 101300., 101300., 101300., 101300., 101300.,
       101300., 101300., 101300., 101300., 101300., 101300., 101300.,
       101300., 101300., 101300., 101300., 101300., 101300.])

# Experimental liquid composition
Xexp = np.array([[0.0028, 0.0118, 0.0137, 0.0144, 0.0176, 0.0222, 0.0246, 0.0302,
        0.0331, 0.0519, 0.053 , 0.0625, 0.0673, 0.0715, 0.0871, 0.126 ,
        0.143 , 0.172 , 0.206 , 0.21  , 0.255 , 0.284 , 0.321 , 0.324 ,
        0.345 , 0.405 , 0.43  , 0.449 , 0.506 , 0.545 , 0.663 , 0.735 ,
        0.804 , 0.917 ],
       [0.9972, 0.9882, 0.9863, 0.9856, 0.9824, 0.9778, 0.9754, 0.9698,
        0.9669, 0.9481, 0.947 , 0.9375, 0.9327, 0.9285, 0.9129, 0.874 ,
        0.857 , 0.828 , 0.794 , 0.79  , 0.745 , 0.716 , 0.679 , 0.676 ,
        0.655 , 0.595 , 0.57  , 0.551 , 0.494 , 0.455 , 0.337 , 0.265 ,
        0.196 , 0.083 ]])

# Experimental vapor composition
Yexp = np.array([[0.032, 0.113, 0.157, 0.135, 0.156, 0.186, 0.212, 0.231, 0.248,
        0.318, 0.314, 0.339, 0.37 , 0.362, 0.406, 0.468, 0.487, 0.505,
        0.53 , 0.527, 0.552, 0.567, 0.586, 0.586, 0.591, 0.614, 0.626,
        0.633, 0.661, 0.673, 0.733, 0.776, 0.815, 0.906],
       [0.968, 0.887, 0.843, 0.865, 0.844, 0.814, 0.788, 0.769, 0.752,
        0.682, 0.686, 0.661, 0.63 , 0.638, 0.594, 0.532, 0.513, 0.495,
        0.47 , 0.473, 0.448, 0.433, 0.414, 0.414, 0.409, 0.386, 0.374,
        0.367, 0.339, 0.327, 0.267, 0.224, 0.185, 0.094]])

datavle = (Xexp, Yexp, Texp, Pexp)

The function ```fit_asso``` optimize the $k_{ij}$ and $l_{ij}$ The function requires initial guesses for the parameters, as well as the mixture object and the equilibria data. Optionally, the ```minimize_options``` option allows modifying the minimizer default parameters.

In [4]:
#initial guess for kij and lij
x0 = np.array([0, 0])
min_options = {'method':'Nelder-Mead'}
fit_asso(x0, mix, datavle = datavle, minimize_options=min_options)

 final_simplex: (array([[-0.0069751 , -0.01521566],
       [-0.00692026, -0.01522137],
       [-0.00707196, -0.01513435]]), array([0.0002247 , 0.00022471, 0.00022471]))
           fun: 0.00022470420148070245
       message: 'Optimization terminated successfully.'
          nfev: 73
           nit: 40
        status: 0
       success: True
             x: array([-0.0069751 , -0.01521566])

If the mixture exhibits other equilibria types you can supply this experimental data to the ``datalle`` or ``datavlle`` parameters.

- ``datalle``: (Xexp, Wexp, Texp, Pexp)
- ``datavlle``: (Xexp, Wexp, Yexp, Texp, Pexp)

You can specify the weights for each objetive function through the following parameters:

- ``weights_vle``: list or array_like, weights for the VLE objective function.
        - weights_vle[0] = weight for Y composition error, default to 1.
        - weights_vle[1] = weight for bubble pressure error, default to 1.
- ``weights_lle``: list or array_like, weights for the LLE objective function.
        - weights_lle[0] = weight for X (liquid 1) composition error, default to 1.
        - weights_lle[1] = weight for W (liquid 2) composition error, default to 1.
- ``weights_vlle``: list or array_like, weights for the VLLE objective function.
        - weights_vlle[0] = weight for X (liquid 1) composition error, default to 1.
        - weights_vlle[1] = weight for W (liquid 2) composition error, default to 1.
        - weights_vlle[2] = weight for Y (vapor) composition error, default to 1.
        - weights_vlle[3] = weight for equilibrium pressure error, default to 1.

Additionally, you can set options to the SciPy's ``minimize`` function using the ``minimize_options`` parameter.

For more information just run:
```fit_asso?```